In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import csv
from geopy.distance import geodesic 

In [2]:
def load_bridge_data(csv_file_path):
    """Load bridge sensor data from CSV file"""
    df = pd.read_csv(r"C:\Users\Rubin\Downloads\Bridge_Sensors_1.csv")
    return df

In [3]:
def calculate_3d_distance(coord1, coord2):
    """Calculate 3D distance considering lat, lon, and altitude"""

    horizontal_dist = geodesic((coord1[0], coord1[1]), (coord2[0], coord2[1])).meters
    

    vertical_dist = abs(coord1[2] - coord2[2])
    

    distance_3d = np.sqrt(horizontal_dist**2 + vertical_dist**2)
    
    return distance_3d

In [4]:
def get_stiffness_value(src_type, tgt_type, distance_m):
    """
    Get stiffness value based on structural element types and distance
    pier: 1.2, girder: 1.0, pylon: 1.5, ground: 0.8
    """

    type_stiffness = {
        'pier': 1.2,
        'girder': 1.0, 
        'pylon': 1.5,
        'ground': 0.8
    }
    

    if ('pier' in [src_type, tgt_type]) and ('girder' in [src_type, tgt_type]):
        return 1.1  
    elif ('pylon' in [src_type, tgt_type]) and ('girder' in [src_type, tgt_type]):
        return 1.3  
    

    if src_type == tgt_type:
        base_stiffness = type_stiffness.get(src_type, 1.0)
      
        distance_factor = max(0.5, 1.0 - (distance_m / 500.0))
        return round(base_stiffness * distance_factor, 2)
    

    src_stiff = type_stiffness.get(src_type, 1.0)
    tgt_stiff = type_stiffness.get(tgt_type, 1.0)
    return round((src_stiff + tgt_stiff) / 2, 2)


In [5]:
def is_valid_connection(src_type, tgt_type, distance_m, k_value):
    """
    Determine if a connection is structurally valid based on type filtering
    More restrictive for k=3,4 to prevent unrealistic long-distance connections
    """
    # For k=2, allow all connections (original behavior)
    if k_value == 2:
        return True
    
    # Define maximum reasonable distances for different connection types
    max_distances = {
        # Same-type connections (sensors on same structural element)
        ('girder', 'girder'): 150.0,  # Girders can be connected along bridge length
        ('pier', 'pier'): 200.0,      # Piers can be far apart
        ('ground', 'ground'): 300.0,  # Ground sensors can be widely spaced
        ('pylon', 'pylon'): 400.0,    # Pylons are typically far apart
        
        # Cross-type connections (structural connections)
        ('pier', 'girder'): 50.0,     # Pier directly supports girder - short distance
        ('girder', 'pier'): 50.0,     # Same as above
        ('pylon', 'girder'): 80.0,    # Pylon supports girder via cables
        ('girder', 'pylon'): 80.0,    # Same as above
        ('ground', 'pier'): 20.0,     # Ground sensor near pier foundation
        ('pier', 'ground'): 20.0,     # Same as above
        ('ground', 'girder'): 100.0,  # Ground sensor monitoring girder from below
        ('girder', 'ground'): 100.0,  # Same as above
        ('ground', 'pylon'): 30.0,    # Ground sensor near pylon foundation
        ('pylon', 'ground'): 30.0,    # Same as above
    }
    
    # Get maximum allowed distance for this connection type
    connection_key = (src_type, tgt_type)
    max_dist = max_distances.get(connection_key, 200.0)  # Default fallback
    
    return distance_m <= max_dist

In [6]:
def get_improved_stiffness_value(src_type, tgt_type, distance_m, k_value):
    """
    Improved stiffness calculation with better structural engineering principles
    """

    if k_value == 2:
        return get_stiffness_value(src_type, tgt_type, distance_m)
    

    base_stiffness = {
        'pier': 1.4,      # Increased - piers are very stiff
        'girder': 1.0,    # Reference value
        'pylon': 1.8,     # Increased - pylons are extremely stiff
        'ground': 0.6     # Decreased - ground has lower stiffness
    }
    
    # Critical structural connections with higher stiffness
    if ('pier' in [src_type, tgt_type]) and ('girder' in [src_type, tgt_type]):
        # Pier-girder: critical load transfer connection
        base_value = 1.4
        # Higher stiffness for closer connections
        distance_factor = max(0.7, 1.2 - (distance_m / 100.0))
        return round(base_value * distance_factor, 2)
    
    elif ('pylon' in [src_type, tgt_type]) and ('girder' in [src_type, tgt_type]):
        # Pylon-girder: main cable support connection
        base_value = 1.6
        distance_factor = max(0.8, 1.3 - (distance_m / 150.0))
        return round(base_value * distance_factor, 2)
    
    elif ('ground' in [src_type, tgt_type]) and (src_type != tgt_type):
        # Ground connections: foundation monitoring
        return 0.7  # Lower stiffness for ground connections
    
    # Same-type connections
    elif src_type == tgt_type:
        base_value = base_stiffness.get(src_type, 1.0)
        
        if src_type == 'girder':
            # Girder-girder: continuous beam behavior
            distance_factor = max(0.6, 1.1 - (distance_m / 200.0))
        elif src_type == 'pier':
            # Pier-pier: independent support elements
            distance_factor = max(0.5, 1.0 - (distance_m / 300.0))
        else:
            # Other same-type connections
            distance_factor = max(0.5, 1.0 - (distance_m / 250.0))
        
        return round(base_value * distance_factor, 2)
    
    # Other mixed connections
    else:
        src_stiff = base_stiffness.get(src_type, 1.0)
        tgt_stiff = base_stiffness.get(tgt_type, 1.0)
        base_value = (src_stiff + tgt_stiff) / 2
        distance_factor = max(0.6, 1.0 - (distance_m / 200.0))
        return round(base_value * distance_factor, 2)

In [7]:
def create_knn_edges(df, k=3):
    """Create edges using K-Nearest Neighbors algorithm with proper 3D distances and type filtering"""
    # Extract coordinates (lat, lon, altitude)
    coords = df[['latitude', 'longitude', 'altitude_m']].values
    node_ids = df['node_id'].tolist()
    
    print(f"\nAnalyzing sensor positions:")
    girder_alts = df[df['type']=='girder']['altitude_m'].values
    pier_alts = df[df['type']=='pier']['altitude_m'].values
    ground_alts = df[df['type']=='ground']['altitude_m'].values
    
    print(f"Girder sensors altitude range: {min(girder_alts):.1f}m - {max(girder_alts):.1f}m")
    print(f"Pier sensors altitude range: {min(pier_alts):.1f}m - {max(pier_alts):.1f}m") 
    print(f"Ground sensors altitude: {ground_alts[0]:.1f}m")
    print(f"Typical girder-pier height difference: {np.mean(girder_alts) - np.mean(pier_alts):.1f}m")
    

    n_nodes = len(df)
    distances_matrix = np.zeros((n_nodes, n_nodes))
    
    for i in range(n_nodes):
        for j in range(n_nodes):
            if i != j:
                dist = calculate_3d_distance(coords[i], coords[j])
                distances_matrix[i][j] = dist
    

    edge_info = []
    edges_set = set()
    
    print(f"\nApplying type filtering for k={k}:")
    print("- Pier-girder connections limited to 50m (direct support)")
    print("- Ground-pier connections limited to 20m (foundation)")
    print("- Girder-girder connections limited to 150m (continuous beam)")
    
    for i in range(n_nodes):
        src_id = node_ids[i]
        src_type = df.iloc[i]['type']
        

        valid_neighbors = []
        for j in range(n_nodes):
            if i != j:
                tgt_type = df.iloc[j]['type']
                distance = distances_matrix[i][j]
                
                if is_valid_connection(src_type, tgt_type, distance, k):
                    valid_neighbors.append((j, distance))
        
    
        valid_neighbors.sort(key=lambda x: x[1])
        selected_neighbors = valid_neighbors[:k]
        
        print(f"\nNode {src_id} ({src_type}) connects to:")
        rejected_count = len([j for j in range(n_nodes) if i != j]) - 1 - len(valid_neighbors)
        if rejected_count > 0 and k > 2:
            print(f"  (Rejected {rejected_count} connections due to type filtering)")
        
        for idx, (j, distance) in enumerate(selected_neighbors):
            tgt_id = node_ids[j]
            tgt_type = df.iloc[j]['type']
            print(f"  {idx+1}. {tgt_id} ({tgt_type}) - {distance:.1f}m")
        
        for j, distance in selected_neighbors:
            tgt_id = node_ids[j]
            tgt_type = df.iloc[j]['type']
            

            edge = tuple(sorted((src_id, tgt_id)))
            if edge not in edges_set:
                edges_set.add(edge)
                
                # Determine edge type
                if src_type == tgt_type:
                    edge_type = src_type
                else:
                    edge_type = "support" 
                
                # Get improved stiffness value
                stiffness = get_improved_stiffness_value(src_type, tgt_type, distance, k)
                
                edge_info.append({
                    'source': edge[0],
                    'target': edge[1],
                    'type': edge_type,
                    'stiffness': stiffness,
                    'distance_m': round(distance, 2),
                    'src_type': src_type,
                    'tgt_type': tgt_type
                })
    
    return edge_info

In [8]:
def analyze_connections(df, edges, k_value):
    """Analyze the types of connections created"""
    print(f"\n=== Connection Analysis (k={k_value}) ===")
    
 
    connection_counts = {}
    for edge in edges:
        edge_type = edge['type']
        connection_counts[edge_type] = connection_counts.get(edge_type, 0) + 1
    
    print("Connection types:")
    for conn_type, count in sorted(connection_counts.items()):
        avg_stiffness = np.mean([e['stiffness'] for e in edges if e['type'] == conn_type])
        avg_distance = np.mean([e['distance_m'] for e in edges if e['type'] == conn_type])
        print(f"  {conn_type}: {count} edges (avg stiffness: {avg_stiffness:.2f}, avg distance: {avg_distance:.1f}m)")
    

    support_connections = [e for e in edges if e['type'] == 'support']
    print(f"\nStructural connections:")
    print(f"  Support connections (cross-type): {len(support_connections)}")
    
    # Analyze critical connections for k>=3
    if k_value >= 3:
        pier_girder = [e for e in support_connections if 
                      ('pier' in [e['src_type'], e['tgt_type']]) and 
                      ('girder' in [e['src_type'], e['tgt_type']])]
        pylon_girder = [e for e in support_connections if 
                       ('pylon' in [e['src_type'], e['tgt_type']]) and 
                       ('girder' in [e['src_type'], e['tgt_type']])]
        
        print(f"  Pier-girder connections: {len(pier_girder)} (critical load transfer)")
        print(f"  Pylon-girder connections: {len(pylon_girder)} (main cable support)")
        
        if pier_girder:
            avg_pier_girder_stiffness = np.mean([e['stiffness'] for e in pier_girder])
            avg_pier_girder_dist = np.mean([e['distance_m'] for e in pier_girder])
            print(f"    Pier-girder avg: {avg_pier_girder_stiffness:.2f} stiffness, {avg_pier_girder_dist:.1f}m distance")
    
    if support_connections:
        print("  Examples of support connections:")
        for conn in support_connections[:8]:  # Show first 8
            print(f"    {conn['source']} ({conn['src_type']}) ↔ {conn['target']} ({conn['tgt_type']}) - {conn['distance_m']:.1f}m, stiffness={conn['stiffness']}")

In [9]:
def save_edges_csv(edges, filename='bridge_edges.csv'):
    """Save edges to CSV file"""
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['source', 'target', 'type', 'stiffness', 'distance_m', 'src_type', 'tgt_type'])
        for edge in edges:
            writer.writerow([edge['source'], edge['target'], edge['type'], 
                           edge['stiffness'], edge['distance_m'], edge['src_type'], edge['tgt_type']])
    print(f"Edges saved to {filename}")

In [10]:
def create_networkx_graph(df, edges):
    """Create NetworkX graph from nodes and edges"""
    G = nx.Graph()
    

    for _, row in df.iterrows():
        G.add_node(row['node_id'], 
                  type=row['type'],
                  pos=(row['longitude'], row['latitude']),
                  altitude=row['altitude_m'])
    

    for edge in edges:
        G.add_edge(edge['source'], edge['target'],
                  type=edge['type'],
                  stiffness=edge['stiffness'],
                  distance_m=edge['distance_m'])
    
    return G

In [11]:
def visualize_knn_bridge_network_with_layout(G, df, k_value, method="KNN"):
    """
    2D Engineering Elevation Visualization
    Bridge layout strictly based on Shin-Nakagawa paper + Delaunay code
    """

    import matplotlib.pyplot as plt
    from matplotlib.lines import Line2D

    # --------------------------------------------------
    # REAL BRIDGE GEOMETRY (meters) — from paper
    # --------------------------------------------------
    spans = [58.9, 75.0, 113.0, 283.9]
    xP = [0]
    for s in spans:
        xP.append(xP[-1] + s)
    bridge_length = xP[-1]

    girder_z = 25.0
    girder_thickness = 2.0
    pier_cap_height = 1.2
    pier_width = 3.0
    pier_cap_width = 8.0
    ground_z = 0.0

    # --------------------------------------------------
    # START FIGURE
    # --------------------------------------------------
    fig, ax = plt.subplots(figsize=(16, 6))

    # --------------------------------------------------
    # DRAW GROUND
    # --------------------------------------------------
    ax.hlines(ground_z, 0, bridge_length, color="black", linewidth=1)

    ax.fill_between(
        [0, bridge_length],
        ground_z,
        girder_z,
        color="black",
        alpha=0.04
    )

    # --------------------------------------------------
    # DRAW PIERS + CAPS (P31–P35)
    # --------------------------------------------------
    for xp in xP:
        # Pier shaft
        ax.fill_between(
            [xp - pier_width / 2, xp + pier_width / 2],
            ground_z,
            girder_z - pier_cap_height,
            color="black",
            alpha=0.12
        )

        # Pier cap
        ax.fill_between(
            [xp - pier_cap_width / 2, xp + pier_cap_width / 2],
            girder_z - pier_cap_height,
            girder_z,
            color="black",
            alpha=0.22
        )

    # --------------------------------------------------
    # DRAW GIRDER
    # --------------------------------------------------
    ax.fill_between(
        [0, bridge_length],
        girder_z,
        girder_z + girder_thickness,
        color="black",
        alpha=0.15
    )

    ax.plot(
        [0, bridge_length],
        [girder_z + girder_thickness, girder_z + girder_thickness],
        color="black",
        linewidth=1.4
    )

    # --------------------------------------------------
    # SNAP PIER SENSORS TO PIER CAPS (as in Delaunay)
    # --------------------------------------------------
    pier_cap_nodes = {39, 40, 35, 31, 27, 28, 23, 24}
    pos2 = {}

    for n in G.nodes():
        x_old, _ = G.nodes[n]["pos"]
        z = G.nodes[n]["altitude"]

        if n in pier_cap_nodes:
            nearest_pier = min(xP, key=lambda p: abs(p - x_old))
            pos2[n] = (nearest_pier, girder_z)
        else:
            pos2[n] = (x_old, z)

    # --------------------------------------------------
    # DRAW EDGES (KNN / MST / etc.)
    # --------------------------------------------------
    for u, v in G.edges():
        x1, y1 = pos2[u]
        x2, y2 = pos2[v]

        ax.plot(
            [x1, x2],
            [y1, y2],
            color="black",
            linewidth=0.8 + G.edges[u, v]["stiffness"],
            alpha=0.6
        )

    # --------------------------------------------------
    # DRAW NODES (ENGINEERING SYMBOLS)
    # --------------------------------------------------
    marker_map = {
        "girder": "s",
        "pier": "o",
        "ground": "^",
        "pylon": "D"
    }

    for n in G.nodes():
        x, y = pos2[n]
        t = G.nodes[n]["type"]

        ax.plot(
            x, y,
            marker=marker_map[t],
            markersize=9,
            markerfacecolor="white",
            color="black"
        )

        ax.text(x, y + 1.0, str(n), fontsize=8, ha="center")

    # --------------------------------------------------
    # AXES & TITLE
    # --------------------------------------------------
    ax.set_xlim(-10, bridge_length + 10)
    ax.set_ylim(0, girder_z + 12)

    ax.set_xlabel("Bridge Horizontal Alignment (m)", fontsize=12)
    ax.set_ylabel("Elevation (m)", fontsize=12)

    ax.set_title(
        f"Bridge Sensor Network – {method} Connectivity\n"
        "Sensor Network on Structural Elevation (Shin-Nakagawa Bridge)",
        fontsize=14
    )

    ax.grid(False)

    # --------------------------------------------------
    # LEGEND (same as Delaunay)
    # --------------------------------------------------
    legend_elements = [
        Line2D([0], [0], marker='s', color='black', markersize=8, label='Girder Sensor'),
        Line2D([0], [0], marker='o', color='black', markersize=8, label='Pier Sensor'),
        Line2D([0], [0], marker='^', color='black', markersize=8, label='Ground Sensor'),
        Line2D([0], [0], color='black', linewidth=2, label='Sensor Connection'),
    ]

    ax.legend(handles=legend_elements, loc="upper left", fontsize=9)

    plt.tight_layout()
    plt.show()


In [12]:
def analyze_graph_properties(G, k_value):
    """Analyze and print graph properties"""
    print(f"=== Graph Network Analysis (k={k_value}) ===")
    print(f"Number of nodes: {G.number_of_nodes()}")
    print(f"Number of edges: {G.number_of_edges()}")
    print(f"Average degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.2f}")
    print(f"Network density: {nx.density(G):.3f}")
    print(f"Is connected: {nx.is_connected(G)}")
    
 
    node_types = {}
    for node in G.nodes():
        node_type = G.nodes[node]['type']
        node_types[node_type] = node_types.get(node_type, 0) + 1
    
    print("\nNode type distribution:")
    for node_type, count in node_types.items():
        print(f"  {node_type}: {count}")
    
 
    edge_types = {}
    stiffness_stats = {}
    
    for edge in G.edges():
        edge_type = G.edges[edge]['type']
        stiffness = G.edges[edge]['stiffness']
        
        edge_types[edge_type] = edge_types.get(edge_type, 0) + 1
        
        if edge_type not in stiffness_stats:
            stiffness_stats[edge_type] = []
        stiffness_stats[edge_type].append(stiffness)
    
    print("\nEdge type distribution:")
    for edge_type, count in edge_types.items():
        avg_stiffness = np.mean(stiffness_stats[edge_type])
        max_stiffness = np.max(stiffness_stats[edge_type])
        min_stiffness = np.min(stiffness_stats[edge_type])
        print(f"  {edge_type}: {count} edges (stiffness: avg={avg_stiffness:.2f}, range={min_stiffness:.2f}-{max_stiffness:.2f})")

In [13]:
# ==================================================
# FINAL KNN PIPELINE (DELAUNAY-STYLE EXECUTION)
# ==================================================

k_values = [2, 3, 4]

for k in k_values:
    print("\n" + "=" * 60)
    print(f"KNN EDGE GENERATION | k = {k}")
    print("=" * 60)

    # Create edges
    edges = create_knn_edges(df, k=k)

    # Save edges
    save_edges_csv(edges, f"bridge_edges_knn_k{k}.csv")

    # Build graph
    G = create_networkx_graph(df, edges)

    # Inject engineering coordinates
    for _, r in df.iterrows():
        n = r["node_id"]
        G.nodes[n]["pos"] = (r["x_eng"], 0.0)
        G.nodes[n]["altitude"] = r["altitude_m"]
        G.nodes[n]["type"] = r["type"]

    analyze_graph_properties(G, k)

    # 🔴 DELAUNAY-STYLE 2D VISUALIZATION
    visualize_knn_bridge_network_2d(G, df, k)

print("\n=== KNN 2D ELEVATION ANALYSIS COMPLETE ===")



KNN EDGE GENERATION | k = 2


NameError: name 'df' is not defined